# The Hangman Game - Theodoros Mamalis

This is an algorithm that plays the game of Hangman efficiently, with approximatly 55% word-guessing accuracy on dictionaries of hard difficulty. This accuracy is achieved through the algorithm's "guess" function. Compared to other implementations, this is a consice and efficient way of guessing the next letter of an unknown word in a dictionary.

When a user plays Hangman, the game first selects a secret word at random from a list. The game then returns a row of underscores (space separated)—one for each letter in the secret word—and asks the user to guess a letter. If the user guesses a letter that is in the word, the word is redisplayed with all instances of that letter shown in the correct positions, along with any letters correctly guessed on previous turns. If the letter does not appear in the word, the user is charged with an incorrect guess. The user keeps guessing letters until either (1) the user has correctly guessed all the letters in the word or (2) the user has made a prespecified number of incorrect guesses.

The algorithm uses a training set of approximately 250,000 dictionary words. The algorithm is tested on an entirely disjoint set of 250,000 dictionary words. This means the words that ultimately the algorithm is tested on do NOT appear in the dictionary that the algorithm is trained on. 

In [248]:
import json
import requests
import random
import string
import secrets
import time
import re
import collections
try:
    from urllib.parse import parse_qs, urlencode, urlparse
except ImportError:
    from urlparse import parse_qs, urlparse
    from urllib import urlencode
    
from string import ascii_lowercase
from nltk import ngrams

from statistics import fmean

In [444]:
class HangmanAPI(object):
    def __init__(self):
        full_dictionary_location = "dictionaries/words_250000_train.txt"
        test_dictionary_location = "dictionaries/words_test_hard.txt"
        self.full_dictionary = self.build_dictionary(full_dictionary_location)        
        self.test_dictionary = self.build_dictionary(test_dictionary_location)
        self.guessed_letters = []
        self.ttry = 0
        self.total_successes = []
        
        self.wrong_letters = []
        
        
        
        full_dictionary_location = "words_250000_train.txt"
        self.full_dictionary_common_letter_sorted = collections.Counter("".join(self.full_dictionary)).most_common()
        
        self.current_dictionary = []
        
        
    def n_gram_probs(self, clean_word, full_dictionary, guessed_letter_list, wrong_letter_list, n):
        ngram = []
        total_count = 0
        letters_list = list(ascii_lowercase)
        
        # The letters that are already guessed are not considered        
        letters_list = [x for x in letters_list if x not in guessed_letter_list]
        letter_count = dict.fromkeys(list(ascii_lowercase),0)
        probabilities = dict.fromkeys(list(ascii_lowercase),0)
        
        # The first guess is made to be always "a"; the letter "e" could also be chosen
        if len(letters_list)==len(ascii_lowercase): 
            probabilities["a"]=1


        
        ## Generate n-grams for each word in the dictionary
        for dict_word in full_dictionary:
            ngram_word = ["".join(k1) for k1 in list(ngrams(dict_word,n=n))]
            ngram.append(ngram_word)

        # tokenize the clean word
        ngram_list = [x for someList in ngram for x in someList]
        counter_list = collections.Counter(ngram_list)
        tokenized_clean_word=list(ngrams(clean_word,n=n))
            
        # for each token find out how many ngrams exist for a given letter
        for tokens_list in tokenized_clean_word:
            if collections.Counter(tokens_list)["."] ==1:
                joined_tokens_dot ="".join(tokens_list)
                for letter in letters_list:
                    joined_tokens_sub=re.sub("[.]",letter,joined_tokens_dot)
                    letter_count[letter]+= counter_list[joined_tokens_sub]

        # calculate probabilities for each letter
        for key, value in letter_count.items():
            probabilities[key]=probabilities[key]+value
        total = sum(probabilities.values())
        if total:
            probabilities = {key: value / total for key, value in probabilities.items()}
            
        return probabilities

        
    def guess(self, word): # word input example: "_ p p _ e "
        ###############################################
        ###############################################

        # clean the word so that we strip away the space characters
        # replace "_" with "." as "." indicates any character in regular expressions
        clean_word = word[::2].replace("_",".")
        total_scores = {}
 
        
        # print the wrong letters guessed so far
        wrong_letters=self.wrong_letters
        for wrong_letter in wrong_letters:    
            print("previous wrong letter guessed: {0}".format(wrong_letter))
        print("wrong_letters_list:",wrong_letters)

        
        # find length of passed word
        len_word = len(clean_word)
        
        # grab current dictionary of possible words from self object, initialize new possible words dictionary to empty
        current_dictionary = self.current_dictionary
        new_dictionary = []
        
        # iterate through all of the words in the old plausible dictionary
        ttry = self.ttry
        
        # use ngrams to calculate probabilities for each letters
        new_dictionary = self.full_dictionary
        probs_six = self.n_gram_probs(clean_word, new_dictionary, self.guessed_letters, wrong_letters, 6)
        probs_five = self.n_gram_probs(clean_word, new_dictionary, self.guessed_letters, wrong_letters, 5)
        probs_four = self.n_gram_probs(clean_word, new_dictionary, self.guessed_letters, wrong_letters, 4)
        probs_tri = self.n_gram_probs(clean_word, new_dictionary, self.guessed_letters, wrong_letters, 3)
        probs_bi = self.n_gram_probs(clean_word, new_dictionary, self.guessed_letters, wrong_letters, 2)
        probs_uni = self.n_gram_probs(clean_word, new_dictionary, self.guessed_letters, wrong_letters, 1)
        wt = [0.23,0.2,0.25,0.18,0.1,0.04]

        # Calculate final scores
        for key in probs_five:
            total_scores[key] = probs_six[key]*wt[0] + probs_five[key]*wt[1] + probs_four[key]*wt[2] + probs_tri[key]*wt[3]+ probs_bi[key]*wt[4] + probs_uni[key]*wt[5] 
        
        # find the letter with the highest score obtained from the ngram procedure and announce it as the guess
        max_value = max(total_scores.values())
        guess_letters = [k for k,v in total_scores.items() if v == max_value]
        guess_letter = guess_letters[0]
        print("best_guess (first guess is always \"a\"):", guess_letter)
        
        return guess_letter

    ##########################################################
    ##########################################################
    
    def build_dictionary(self, dictionary_file_location):
        text_file = open(dictionary_file_location,"r")
        full_dictionary = text_file.read().splitlines()
        text_file.close()
        return full_dictionary
                
    def start_game(self, correct_word=None, see_actual=False, train_test="train", verbose=True, tries_remain=6):
        # reset guessed letters to empty set and current plausible dictionary to the full dictionary
        self.guessed_letters = []
        self.current_dictionary = self.full_dictionary
        self.wrong_letters = []
            
        tries = tries_remain

        if not correct_word:
            if train_test == 'train':
                correct_word = random.choice(self.full_dictionary)
            else:
                correct_word = random.choice(self.test_dictionary)

        if verbose and see_actual:
            print('The word is {}'.format(actual_word))


        characters = [i for i in actual_word]
        masked = ['_' for letter in characters]
        word = ' '.join(masked)

        while tries_remain>0:
            self.ttry=tries_remain

            # save the wrong guesses
            if tries - tries_remain > 0:
                self.wrong_letters.append(guess_letter)
                tries = tries_remain     

            if verbose:
                print("{} tries remaining".format(self.tries_remain))
                print(word)
                print('\n')

            # get guessed letter from user code                
            guess_letter = self.guess(word)

            # append guessed letter to guessed letters field in hangman object                
            self.guessed_letters.append(guess_letter)


            if verbose:
                print("Guessing letter: {0}".format(guess_letter))
                print('\n')

            if guess_letter in characters:
                indices = [i for i, x in enumerate(characters) if x == guess_letter]

                for index in indices:
                    masked[index] = guess_letter

                word = ' '.join(masked)

                if '_' not in word:
                    if verbose:
                        print(word)
                        print('You win!')
                        self.total_successes.append(1)
                    break

            else:
                if verbose:
                    print("Bad guess")
                    print('\n')
                self.tries_remain = self.tries_remain - 1

        if '_' in word:
            if verbose:
                print("You lose!")
            self.total_successes.append(0)


# Playing practice games:

In [443]:
game=HangmanAPI()

number_of_games = 100000
for i in range(number_of_games):
    print('Playing ', i, ' th game')
    game.start_game((verbose=True, train_test="train", number_of_games=100000))

    
practice_success_rate = statistics.mean(game.total_successes)
print('run %d practice games out of an allotted 100,000. practice success rate so far = %.3f' % (number_of_games, practice_success_rate))


Successfully start a new game! Game ID: 59fefe53a89a. # of tries remaining: 6. Word: _ _ _ _ _ _ _ _ _ .
wrong_letters_list: []
best_guess (first guess is always "a"): a
Guessing letter: a
Sever response: {'game_id': '59fefe53a89a', 'status': 'ongoing', 'tries_remains': 5, 'word': '_ _ _ _ _ _ _ _ _ '}
previous wrong letter guessed: a
wrong_letters_list: ['a']
best_guess (first guess is always "a"): e
Guessing letter: e
Sever response: {'game_id': '59fefe53a89a', 'status': 'ongoing', 'tries_remains': 5, 'word': '_ _ _ _ _ _ e _ _ '}
previous wrong letter guessed: a
wrong_letters_list: ['a']
best_guess (first guess is always "a"): r
Guessing letter: r
Sever response: {'game_id': '59fefe53a89a', 'status': 'ongoing', 'tries_remains': 4, 'word': '_ _ _ _ _ _ e _ _ '}
previous wrong letter guessed: a
previous wrong letter guessed: r
wrong_letters_list: ['a', 'r']
best_guess (first guess is always "a"): n
Guessing letter: n
Sever response: {'game_id': '59fefe53a89a', 'status': 'ongoing', 'tr

#  Playing test games:


In [447]:
game=HangmanAPI()

number_of_games = 1000

for i in range(number_of_games):
    print('Playing ', i, ' th game')
    game.start_game(verbose=True, train_test="test")
    

Playing  0  th game
wrong_letters_list: []
best_guess (first guess is always "a"): a
wrong_letters_list: []
best_guess (first guess is always "a"): n
wrong_letters_list: []
best_guess (first guess is always "a"): i
previous wrong letter guessed: i
wrong_letters_list: ['i']
best_guess (first guess is always "a"): e
previous wrong letter guessed: i
wrong_letters_list: ['i']
best_guess (first guess is always "a"): t
previous wrong letter guessed: i
previous wrong letter guessed: t
wrong_letters_list: ['i', 't']
best_guess (first guess is always "a"): m
previous wrong letter guessed: i
previous wrong letter guessed: t
previous wrong letter guessed: m
wrong_letters_list: ['i', 't', 'm']
best_guess (first guess is always "a"): d
previous wrong letter guessed: i
previous wrong letter guessed: t
previous wrong letter guessed: m
previous wrong letter guessed: d
wrong_letters_list: ['i', 't', 'm', 'd']
best_guess (first guess is always "a"): l
previous wrong letter guessed: i
previous wrong lett

HangmanAPIError: {'error': 'You have reached 1000 of games', 'status': 'denied'}

In [448]:
# Returns the total number of games, and number of wins.

success_rate = statistics.mean(game.total_recorded_successes)
print('overall success rate = %.3f' % success_rate)

overall success rate = 0.550
